In [127]:
import pandas as pd
import numpy as np
from typing import List, Optional, Tuple

In [217]:
shrooms = pd.read_csv("https://users.csc.calpoly.edu/~dekhtyar/466-Fall2023/data/MUSHROOM/agaricus-lepiota.csv", skiprows=[1],
                      header=0)
classvar = shrooms.iloc[0, 0]

shrooms = shrooms.drop(0, axis=0)

C = shrooms[classvar]
D = shrooms.drop(classvar, axis=1)
A = list(D.columns)

In [218]:
shrooms

,Class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
1,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
2,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
3,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
4,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
5,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8120,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8121,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8122,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8123,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


In [180]:
C.unique()[0]

'p'

In [181]:
def entropy(D: pd.DataFrame, C: pd.Series,
            A_i: Optional[str] = None, x: Optional[float] = None) -> float:
    e = 0
    if A_i is None and x is None:
        e = C.value_counts(normalize=True).apply(
            lambda pr: -pr * np.log2(pr)).sum()
    elif x is None:
        prop = D[A_i].value_counts(normalize=True)
        for a in list(prop.index):
           e += prop.loc[a] * entropy(D[D[A_i] == a], C[D[A_i] == a])
    else:
        # TODO: Implement this in part 2
        pass

    return e

In [206]:
entropy(D, C, A_i="veil-type")

0.9990678968724603

In [209]:
D['veil-type'].value_counts()

veil-type
p    8124
Name: count, dtype: int64

In [207]:
np.log2(2)

1.0

In [185]:
entropy(D, C)

0.9990678968724603

In [186]:
# TODO: Implement this in part 2
# def findBestSplit(A_i: str, D: pd.DataFrame) -> float:
#     pass

In [187]:
def find_most_frequent_label(C: pd.Series) -> Tuple[str, float]:
    prop = C.value_counts(normalize=True)
    return str(prop.idxmax()), prop.max()

In [204]:
def selectSplittingAttribute(A: List[str], D: pd.DataFrame, C: pd.Series,
                             threshold: float, gratio: bool = False) -> Optional[str]:
    p = {}
    gain = {}
    p[0] = entropy(D, C)
    for A_i in A:
        # TODO: Implement this in part 2
        # if pd.api.types.is_numeric_dtype(D.dtypes[A_i]):
        #     x = findBestSplit(A_i, D)
        #     p[A_i] = entropy(D, C, A_i, x)
        # else:
        #     p[A_i] = entropy(D, C, A_i)
        # TODO: Delete next line once part 2 implemented
        p[A_i] = entropy(D, C, A_i)
        gain[A_i] = p[0] - p[A_i]
        if gratio:
            denom = D[A_i].value_counts(normalize=True).apply(
                lambda pr: -pr * np.log2(pr)).sum()
            if gain[A_i] != 0 and denom != 0:
                gain[A_i] = gain[A_i] / denom
            elif gain[A_i] == 0:
                gain[A_i] = 0
            elif denom == 0:
                gain[A_i] = np.infty
            print(A_i, gain[A_i])
    best = max(gain, key=gain.get)
    if gain[best] > threshold:
        return best
    else:
        return None

In [205]:
selectSplittingAttribute(A, D, C, 0.1, gratio=True)

cap-shape 0.0295220698859018
cap-surface 0.01814692451164122
cap-color 0.014361447780696823
bruises 0.19644043129956845
odor 0.3906481565033008
gill-attachment 0.0818178118172925
gill-spacing 0.15815426132075966
gill-size 0.25794648736352677
gill-color 0.13759668648296522
stalk-shape 0.007616338689020934
stalk-root 0.07395690261310016
stalk-surface-above-ring 0.2331240725567335
stalk-surface-below-ring 0.1943304679865192
stalk-color-above-ring 0.13106361757552198
stalk-color-below-ring 0.12204028298851571
veil-type 0
veil-color 0.12136782986327024
ring-number 0.09140595216694161
ring-type 0.207163744271962
spore-print-color 0.21818217753240354
population 0.10080772885489371
habitat 0.0689455094456155


'odor'

In [152]:
def C45(D: pd.DataFrame, A: List[str], C: pd.Series,
        threshold: float, gratio: bool = False) -> dict:
    dataset = "something.csv"
    T = {"dataset": dataset}
    if len(C.unique()) == 1:
        T["leaf"] = {"decision": C.unique()[0], 'p': 1}
    elif len(A) == 0:
        c, p = find_most_frequent_label(C)
        T["leaf"] = {"decision": c, "p": p}
    else:
        A_g = selectSplittingAttribute(A, D, C, threshold, gratio=gratio)
        if A_g is None:
            c, p = find_most_frequent_label(C)
            T["leaf"] = {"decision": c, "p": p}
        else:
            r = {"var": A_g, "edges": []}
            T["node"] = r
            for v in list(D[A_g].unique()):
                D_v = D[D[A_g] == v]
                C_v = C[D[A_g] == v]
                if len(D_v.index) != 0:
                    # TODO: Implement this in part 2
                    # if D.dtypes[A_g] == object:
                    #     A_v = A[:]
                    #     A_v.remove(A_g)
                    # TODO: Delete next 2 lines once part 2 implemented
                    A_v = A[:]
                    A_v.remove(A_g)
                    T_v = C45(D_v, A_v, C_v, threshold)
                    new_edge = {"value": v}
                    if "node" in T_v.keys():
                        new_edge["node"] = T_v["node"]
                    elif "leaf" in T_v.keys():
                        new_edge["leaf"] = T_v["leaf"]
                    r["edges"].append(
                        {"edge": new_edge}
                    )
                else:
                    c, p = find_most_frequent_label(C)
                    r["edges"].append(
                        {"edge": {"value": v,
                                  "leaf": {"decision": c, "p": p}}}
                    )
    return T

In [162]:
C45(D, A, C, 0.142)

{'dataset': 'something.csv',
 'node': {'var': 'odor',
  'edges': [{'edge': {'value': 'p', 'leaf': {'decision': 'p', 'p': 1}}},
   {'edge': {'value': 'a', 'leaf': {'decision': 'e', 'p': 1}}},
   {'edge': {'value': 'l', 'leaf': {'decision': 'e', 'p': 1}}},
   {'edge': {'value': 'n',
     'node': {'var': 'spore-print-color',
      'edges': [{'edge': {'value': 'n', 'leaf': {'decision': 'e', 'p': 1}}},
       {'edge': {'value': 'k', 'leaf': {'decision': 'e', 'p': 1}}},
       {'edge': {'value': 'w',
         'node': {'var': 'habitat',
          'edges': [{'edge': {'value': 'w',
             'leaf': {'decision': 'e', 'p': 1}}},
           {'edge': {'value': 'l',
             'node': {'var': 'cap-color',
              'edges': [{'edge': {'value': 'c',
                 'leaf': {'decision': 'e', 'p': 1}}},
               {'edge': {'value': 'n', 'leaf': {'decision': 'e', 'p': 1}}},
               {'edge': {'value': 'w', 'leaf': {'decision': 'p', 'p': 1}}},
               {'edge': {'value': 'y', 

In [32]:
T = {}
r = {"object": [1]}
T["for"] = r
r["object"].append(2)
print(T)

{'for': {'object': [1, 2]}}


In [36]:
type(C.value_counts().idxmax())

str

In [47]:
df = pd.DataFrame({"A": [1, 2, 3]})
pd.api.types.is_string_dtype(df['A'])

False

In [64]:
len(df.drop('A', axis=1).columns)

0

In [168]:
nursery = pd.read_csv("https://users.csc.calpoly.edu/~dekhtyar/466-Fall2023/data/NURSERY/nursery.csv", skiprows=[1, 2], header=0)
nursery

,parents,has_nurs,form,children,housing,finance,social,health,class
0,usual,proper,complete,1,convenient,convenient,nonprob,recommended,recommend
1,usual,proper,complete,1,convenient,convenient,nonprob,priority,priority
2,usual,proper,complete,1,convenient,convenient,nonprob,not_recom,not_recom
3,usual,proper,complete,1,convenient,convenient,slightly_prob,recommended,recommend
4,usual,proper,complete,1,convenient,convenient,slightly_prob,priority,priority
...,...,...,...,...,...,...,...,...,...
12955,great_pret,very_crit,foster,more,critical,inconv,slightly_prob,priority,spec_prior
12956,great_pret,very_crit,foster,more,critical,inconv,slightly_prob,not_recom,not_recom
12957,great_pret,very_crit,foster,more,critical,inconv,problematic,recommended,spec_prior
12958,great_pret,very_crit,foster,more,critical,inconv,problematic,priority,spec_prior


In [171]:
D = nursery.drop('class', axis=1)
A = list(D.columns)
C = nursery['class']

In [176]:
C45(D, A, C, 0.3)

{'dataset': 'something.csv',
 'node': {'var': 'health',
  'edges': [{'edge': {'value': 'recommended',
     'leaf': {'decision': 'priority', 'p': 0.5583333333333333}}},
   {'edge': {'value': 'priority',
     'node': {'var': 'has_nurs',
      'edges': [{'edge': {'value': 'proper',
         'node': {'var': 'parents',
          'edges': [{'edge': {'value': 'usual',
             'leaf': {'decision': 'priority', 'p': 1}}},
           {'edge': {'value': 'pretentious',
             'leaf': {'decision': 'priority', 'p': 1}}},
           {'edge': {'value': 'great_pret',
             'leaf': {'decision': 'spec_prior', 'p': 0.65625}}}]}}},
       {'edge': {'value': 'less_proper',
         'node': {'var': 'parents',
          'edges': [{'edge': {'value': 'usual',
             'leaf': {'decision': 'priority', 'p': 1}}},
           {'edge': {'value': 'pretentious',
             'leaf': {'decision': 'priority', 'p': 1}}},
           {'edge': {'value': 'great_pret',
             'leaf': {'decision': 's